# Tool 수정 사항 검증 테스트

이 노트북은 아래 두 가지 수정 내용이 올바르게 적용됐는지 확인합니다.

---

## 수정 1: `action()` 개선

| 항목 | 수정 전 | 수정 후 |
|------|---------|--------|
| LLM에 전달하는 세계 정보 | `world_state` 전체 (모든 NPC 상세 정보 포함) | `world_snapshot`만 (필요한 핵심 정보만) |
| 프롬프트 구조 | 단일 `prompt` 문자열 | `system_prompt` + `user_prompt` 분리 |

## 수정 2: `use()` 개선

| 항목 | 수정 전 | 수정 후 |
|------|---------|--------|
| 아이템 효과 결정 방식 | LLM 호출 | 룰 엔진(`ItemUseResolver`) 사용 |
| 반환값 | `event_description`, `state_delta` | + `item_id`, `item_use_result` 추가 |
| 인벤토리 변경 | 미지원 | `state_delta.inventory_add/remove` |
| NPC 상태 변경 | 미지원 | `state_delta.npc_status_changes` (수면제 등) |

---

## 전체 흐름

```
사용자 입력 (자연어)
       ↓
DayController.process()
       ↓
call_tool()  ← LLM이 어떤 tool을 쓸지 결정
       ↓                        ↓                    ↓
   interact()              action()              use()
   [NPC 대화]          [일반 행동/조사]       [아이템 사용]
   LLM 호출             LLM 호출             룰 엔진 (No LLM)
       ↓                    ↓                    ↓
   ToolResult          ToolResult           ToolResult
   - npc_response      - event_desc         - event_desc
   - state_delta       - state_delta        - state_delta
   - npc_id                                 - item_id
                                            - item_use_result
       ↓                    ↓                    ↓
             state_delta 가 WorldState에 반영됨
             (npc_stats, inventory, npc_status_changes 등)
```

## 0. Setup

In [ ]:
import sys
import logging
from pathlib import Path
from dotenv import load_dotenv

ROOT = Path.cwd()
while not (ROOT / "app").exists() and ROOT != ROOT.parent:
    ROOT = ROOT.parent

if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

load_dotenv(ROOT / ".env")
logging.basicConfig(level=logging.WARNING, format="%(name)s | %(message)s")

print(f"Project root: {ROOT}")

In [ ]:
from app.loader import ScenarioLoader
from app.schemas import WorldStatePipeline, NPCState

loader = ScenarioLoader(ROOT / "scenarios")
assets = loader.load("coraline")

# 테스트용 WorldState - 모든 NPC가 있는 상태
world = WorldStatePipeline(
    turn=3,
    npcs={
        "button_mother": NPCState(npc_id="button_mother", stats={"trust": 5, "fear": 0, "suspicion": 3}),
        "button_father": NPCState(npc_id="button_father", stats={"trust": 4, "fear": 0, "suspicion": 4}),
        "button_daughter": NPCState(npc_id="button_daughter", stats={"trust": 6, "fear": 0, "suspicion": 2}),
    },
    inventory=[],
    vars={
        "humanity": 8,
        "total_suspicion": 3,
        "day": 2,
        "suspicion_level": 3,
        "node_id": "day_exploration",
        "current_phase": "day",   # 수면제 테스트 때 'evening_prep'으로 바꿀 예정
    },
    flags={},
)

print(f"시나리오: {assets.scenario.get('title')}")
print(f"NPC: {list(world.npcs.keys())}")
print(f"초기 인벤토리: {world.inventory}")
print(f"현재 vars: {world.vars}")

---
## 검증 1: `action()` — world_snapshot 구조 확인

### 왜 이게 중요한가?

수정 전에는 LLM에게 `world_state` 전체를 넘겼습니다. 이러면:
- NPC 메모리 스트림, 대화 기록 등 불필요한 데이터가 LLM 프롬프트에 들어감
- 토큰 낭비 + 프롬프트가 길어져 응답 품질 저하

수정 후에는 `_build_world_snapshot()`으로 핵심 정보만 추출합니다.

In [ ]:
# _build_world_snapshot() 직접 테스트 (LLM 호출 없음)
from app.tools import _build_world_snapshot

snapshot = _build_world_snapshot(world, assets)

print("=== world_snapshot (LLM에 실제로 전달되는 정보) ===")
for k, v in snapshot.items():
    print(f"  {k}: {v}")

print()
print("=== 전체 world_state (원본) ===")
print(f"  turn: {world.turn}")
print(f"  vars: {world.vars}")
print(f"  flags: {world.flags}")
print(f"  inventory: {world.inventory}")
print(f"  npcs (요약):")
for npc_id, npc in world.npcs.items():
    print(f"    {npc_id}: stats={npc.stats}, memory keys={list(npc.memory.keys())}")

In [ ]:
# world_snapshot이 올바른 키만 포함하는지 검증
EXPECTED_SNAPSHOT_KEYS = {"day", "turn", "suspicion_level", "player_humanity", "flags", "node_id", "inventory", "genre", "tone"}
UNEXPECTED_KEYS_IN_SNAPSHOT = {"npcs", "locks"}  # 전체 NPC 상세 정보, 락 정보 등은 없어야 함

actual_keys = set(snapshot.keys())

print("=== world_snapshot 키 검증 ===")
print(f"포함된 키: {sorted(actual_keys)}")
print()

# 검증 1: 필요한 키가 있는가
missing = EXPECTED_SNAPSHOT_KEYS - actual_keys
if missing:
    print(f"[FAIL] 누락된 키: {missing}")
else:
    print(f"[PASS] 필요한 키가 모두 존재함")

# 검증 2: 불필요한 키(전체 NPC 상세 정보 등)가 없는가
unwanted = UNEXPECTED_KEYS_IN_SNAPSHOT & actual_keys
if unwanted:
    print(f"[FAIL] 불필요한 대형 데이터가 포함됨: {unwanted}")
else:
    print(f"[PASS] 불필요한 NPC 상세 정보가 제외됨 (NPC 메모리, 락 정보 등 없음)")

print()
print("[결론] world_snapshot은 전체 world_state 대비 약 ", end="")
import json
snapshot_size = len(json.dumps(snapshot))
full_size = len(json.dumps(world.model_dump()))
print(f"{snapshot_size}/{full_size} 바이트 = {snapshot_size/full_size*100:.1f}% 크기만 LLM에 전달")

---
## 검증 2: `build_action_prompt()` — system/user 프롬프트 분리 확인

### 왜 분리해야 하는가?

LLM API는 `system_prompt`와 `user_prompt`를 분리해서 받습니다.
- **system_prompt**: "당신은 내러티브 엔진입니다. 규칙: ..." → LLM의 역할/규칙 정의
- **user_prompt**: 실제 게임 상태 + 플레이어 행동 → LLM이 처리할 구체적 입력

분리하지 않으면 역할 정의가 매 요청마다 섞여서 응답 품질이 떨어질 수 있습니다.

In [ ]:
# build_action_prompt() 반환값 구조 검증 (LLM 호출 없음)
from app.llm.prompt import build_action_prompt, SYSTEM_PROMPT_ACTION

snapshot = _build_world_snapshot(world, assets)
test_action = "복도 끝에 있는 잠긴 문을 조사한다"

result = build_action_prompt(
    action=test_action,
    world_snapshot=snapshot,
    npc_context=assets.export_for_prompt(),
    assets=assets,
)

print("=== build_action_prompt() 반환 타입 검증 ===")
print(f"반환 타입: {type(result)}")
print(f"반환 길이 (튜플 요소 수): {len(result)}")
assert isinstance(result, tuple), "[FAIL] tuple을 반환해야 합니다"
assert len(result) == 2, "[FAIL] (system_prompt, user_prompt) 2개 요소여야 합니다"
print("[PASS] (system_prompt, user_prompt) tuple 반환 확인")

system_prompt, user_prompt = result

print(f"\nsystem_prompt 길이: {len(system_prompt)}자")
print(f"user_prompt 길이: {len(user_prompt)}자")

In [ ]:
# 프롬프트 내용 검증
print("=== system_prompt (역할 정의 / 규칙) ===")
print(system_prompt)

print()
print("=" * 60)
print("=== user_prompt (게임 상태 + 플레이어 행동) ===")
print(user_prompt)

In [ ]:
# system/user 분리가 잘 됐는지 의미적으로 확인
print("=== 역할 분리 의미 검증 ===")

# system_prompt에는 게임 상태(turn, suspicion 등)가 없어야 함
state_info_in_system = any(word in system_prompt for word in ["turn:", "suspicion_level:", "player_humanity:"])
print(f"system_prompt에 게임 상태 정보 포함 여부: {state_info_in_system}")
if not state_info_in_system:
    print("[PASS] system_prompt는 역할/규칙만 포함 (게임 상태 없음)")
else:
    print("[WARN] system_prompt에 게임 상태가 혼합되어 있음")

# user_prompt에는 행동 내용이 있어야 함
action_in_user = test_action in user_prompt
print(f"\nuser_prompt에 행동 내용 포함 여부: {action_in_user}")
if action_in_user:
    print("[PASS] user_prompt에 플레이어 행동이 포함됨")
else:
    print("[FAIL] user_prompt에 행동 내용이 없음")

# user_prompt에는 세계 상태가 있어야 함
world_info_in_user = "세계 상태" in user_prompt or "[세계" in user_prompt
print(f"\nuser_prompt에 세계 상태 포함 여부: {world_info_in_user}")
if world_info_in_user:
    print("[PASS] user_prompt에 게임 상태(세계 스냅샷) 포함됨")
else:
    print("[FAIL] user_prompt에 게임 상태가 없음")

---
## 검증 3: `use()` — 룰 엔진 기반 아이템 사용

### 아이템 사용 흐름

```
use(item, action, target)
       ↓
ItemUseResolver.resolve()  ← LLM 없음! 룰 엔진만
       ↓
  Step 1: Validate  (아이템 존재? 인벤토리에 있음? 조건 충족?)
       ↓
  Step 2: Simulate  (효과 계산 - EffectApplicator)
       ↓
  Step 3: Commit    (소비 여부 결정)
       ↓
  ItemUseResult
  - state_delta: {npc_stats, npc_status_changes, inventory_remove, vars, ...}
  - status_effects: [StatusEffect(duration 있는 효과)]
  - item_consumed: bool
```

### 테스트 케이스 3개
1. **warm_black_tea** (consumable) - 마시면 소비되고 `vars` 변화
2. **real_family_photo** (lore_clue) - NPC에게 사용하면 `npc_stats` 변화, 소비 안 됨
3. **industrial_sedative** (tool) - 모든 NPC를 수면 상태로 → `npc_status_changes`

In [ ]:
# use()는 _tool_context에서 world_state, assets를 가져오므로 먼저 설정
from app.tools import set_tool_context
from app.llm import get_llm

llm_engine = get_llm()
set_tool_context(world_state=world, assets=assets, llm_engine=llm_engine)
print("tool context 설정 완료")

In [ ]:
# 테스트 3-1: warm_black_tea (consumable)
# - allowed_when: "true" → 항상 사용 가능
# - effects: player.stamina +20, player.humanity -5
# - 소비: type=consumable → inventory_remove에 추가됨

import copy
from app.tools import use

# 인벤토리에 홍차 추가
world_tea = copy.deepcopy(world)
world_tea.inventory = ["warm_black_tea"]
set_tool_context(world_state=world_tea, assets=assets, llm_engine=llm_engine)

tea_result = use(item="warm_black_tea", action="홍차를 마신다")

print("=== 테스트 3-1: warm_black_tea (소비 아이템) ===")
print()
print(f"event_description: {tea_result['event_description']}")
print()
print(f"item_id: {tea_result['item_id']}")  # 수정 후 추가된 필드
print()
print("state_delta 상세:")
sd = tea_result['state_delta']
print(f"  vars (플레이어 스탯 변화): {sd.get('vars', {})}")
print(f"  inventory_remove (소비로 인한 제거): {sd.get('inventory_remove', [])}")
print(f"  inventory_add: {sd.get('inventory_add', [])}")
print(f"  npc_stats: {sd.get('npc_stats', {})}")
print()
print("item_use_result 요약:")
iur = tea_result['item_use_result']
print(f"  success: {iur['success']}")
print(f"  item_consumed: {iur['item_consumed']}")
print(f"  effects_applied: {iur['effects_applied']}")

# 검증
assert tea_result['item_id'] == 'warm_black_tea', "[FAIL] item_id 필드 없음"
assert 'warm_black_tea' in sd.get('inventory_remove', []), "[FAIL] consumable이므로 inventory_remove에 있어야 함"
assert iur['item_consumed'] == True, "[FAIL] item_consumed가 True여야 함"
print()
print("[PASS] consumable 아이템은 사용 후 inventory_remove에 추가됨")

In [ ]:
# 테스트 3-2: real_family_photo (lore_clue, 소비되지 않음)
# - allowed_when: "npc.target.id != 'stepmother'" → stepmother 제외한 NPC에게만 사용 가능
# - effects: npc.target.humanity +30, npc.target.affection -10
# - 소비 안 됨: type=lore_clue

world_photo = copy.deepcopy(world)
world_photo.inventory = ["real_family_photo"]
set_tool_context(world_state=world_photo, assets=assets, llm_engine=llm_engine)

# button_father에게 사용 (조건 통과 → stepmother가 아니므로)
photo_result = use(
    item="real_family_photo",
    action="아빠에게 진짜 가족사진을 보여준다",
    target="button_father"
)

print("=== 테스트 3-2: real_family_photo (NPC 스탯 변화) ===")
print()
print(f"event_description: {photo_result['event_description']}")
print()
print("state_delta 상세:")
sd = photo_result['state_delta']
print(f"  npc_stats (NPC 스탯 변화): {sd.get('npc_stats', {})}")
print(f"  inventory_remove: {sd.get('inventory_remove', [])}")
print()
print("item_use_result 요약:")
iur = photo_result['item_use_result']
print(f"  success: {iur['success']}")
print(f"  item_consumed: {iur['item_consumed']}  ← lore_clue는 소비 안 됨")
print(f"  notes: {iur['notes']}")

# 검증
assert 'real_family_photo' not in sd.get('inventory_remove', []), "[FAIL] lore_clue는 소비되면 안 됨"
assert iur['item_consumed'] == False, "[FAIL] lore_clue는 item_consumed=False여야 함"
print()
print("[PASS] lore_clue 아이템은 사용 후에도 소비되지 않음")

In [ ]:
# 테스트 3-3: industrial_sedative (수면제) — npc_status_changes 검증
#
# 수면제의 효과:
#   effects:
#     - { type: set_state, target: "npc.all.status", value: "sleeping", duration: 3 }
#
# 이 효과는 모든 NPC의 status를 'sleeping'으로 바꾸고,
# 3턴 후 원래 상태로 돌아오는 StatusEffect를 생성합니다.
#
# 사용 조건: allowed_when: "system.phase == 'evening_prep'"
# → world.vars["current_phase"] = "evening_prep" 으로 세팅해야 통과

world_sedative = copy.deepcopy(world)
world_sedative.inventory = ["industrial_sedative"]
world_sedative.vars["current_phase"] = "evening_prep"  # 조건 충족
set_tool_context(world_state=world_sedative, assets=assets, llm_engine=llm_engine)

sedative_result = use(
    item="industrial_sedative",
    action="새엄마 음식에 수면제를 탄다"
)

print("=== 테스트 3-3: industrial_sedative (NPC 전체 수면 상태) ===")
print()
print(f"event_description: {sedative_result['event_description']}")
print()
print("state_delta 상세:")
sd = sedative_result['state_delta']
print(f"  npc_status_changes: {sd.get('npc_status_changes', {})}")
print(f"    → 위 NPC들이 'sleeping' 상태로 변경됨")
print(f"  inventory_remove: {sd.get('inventory_remove', [])}")
print(f"    → type=tool이므로 소비됨")
print()
print("item_use_result 상세:")
iur = sedative_result['item_use_result']
print(f"  success: {iur['success']}")
print(f"  item_consumed: {iur['item_consumed']}")
print(f"  notes: {iur['notes']}")
print()
print("status_effects (지속 효과 목록):")
for se in iur['status_effects']:
    print(f"  - NPC: {se['target_npc_id']}")
    print(f"    applied_status: {se['applied_status']} (수면 상태로 변경)")
    print(f"    expires_at_turn: {se['expires_at_turn']} (현재 턴 {world_sedative.turn} + duration 3 = {world_sedative.turn + 3})")
    print(f"    original_status: {se['original_status']} (만료 시 복구될 상태)")

# 검증
assert iur['success'] == True, "[FAIL] 수면제 사용이 실패함"
npc_status_changes = sd.get('npc_status_changes', {})
assert len(npc_status_changes) > 0, "[FAIL] npc_status_changes가 비어있음"
for npc_id, status in npc_status_changes.items():
    assert status == "sleeping", f"[FAIL] {npc_id}의 상태가 'sleeping'이 아님: {status}"
assert 'industrial_sedative' in sd.get('inventory_remove', []), "[FAIL] 수면제가 소비되어야 함"
print()
print(f"[PASS] 수면제가 {len(npc_status_changes)}명의 NPC를 sleeping 상태로 변경")
print(f"[PASS] 수면제가 inventory_remove에 추가됨 (소비)")
print(f"[PASS] StatusEffect {len(iur['status_effects'])}개 생성 (DayController가 만료 관리)")

### StatusEffect란?

`status_effects`는 **지속 시간이 있는 NPC 상태 변화**입니다.

```python
StatusEffect(
    target_npc_id = "button_mother",
    applied_status = "sleeping",    # 지금 적용할 상태
    original_status = "alive",      # 만료되면 돌아올 상태
    expires_at_turn = 6,            # 현재 턴(3) + duration(3)
)
```

`DayController.process()`는 use() 결과에서 `status_effects`를 꺼내서
`StatusEffectManager`에 등록합니다. 이후 매 턴마다 만료 체크를 합니다.

---
## 검증 4: `StateDelta` 스키마 — 아이템 관련 필드 확인

수정 사항에서 요구한 StateDelta 필드들이 실제로 존재하는지 확인합니다.

In [ ]:
from app.schemas.game_state import StateDelta
import inspect

print("=== StateDelta 스키마 필드 목록 ===")
print()
for field_name, field_info in StateDelta.model_fields.items():
    print(f"  {field_name}: {field_info.annotation}")
    if hasattr(field_info, 'default'):
        print(f"    default: {field_info.default}")

print()
print("=== 수정 사항에서 요구한 필드 검증 ===")
required_fields = [
    ("inventory_add",     "아이템 획득"),
    ("inventory_remove",  "아이템 사용/소비"),
    ("npc_status_changes","NPC 상태 변경 (수면, 사망 등)"),
    ("npc_stats",         "NPC 수치 스탯 변화"),
    ("vars",              "월드 변수 변화 (인간성 등)"),
    ("flags",             "이벤트 플래그 변화"),
]

all_ok = True
for field, desc in required_fields:
    exists = field in StateDelta.model_fields
    status = "[PASS]" if exists else "[FAIL]"
    print(f"  {status} {field:25s} ← {desc}")
    if not exists:
        all_ok = False

print()
if all_ok:
    print("StateDelta에 필요한 모든 필드가 존재합니다.")
else:
    print("일부 필드가 없습니다. 스키마를 확인하세요.")

In [ ]:
# merge_deltas()로 여러 효과가 올바르게 합쳐지는지도 확인
from app.schemas.game_state import merge_deltas

delta1 = {
    "npc_stats": {"button_mother": {"trust": -2}},
    "inventory_add": ["dog_treat"],
    "vars": {"total_suspicion": 3},
}
delta2 = {
    "npc_status_changes": {"button_mother": "sleeping", "button_father": "sleeping"},
    "inventory_remove": ["industrial_sedative"],
    "vars": {"total_suspicion": 2},  # 누적됨 → 5가 되어야
}

merged = merge_deltas(delta1, delta2)

print("=== merge_deltas() 검증 ===")
print()
print("delta1:", delta1)
print("delta2:", delta2)
print()
print("merged 결과:")
for k, v in merged.items():
    if v and v != {} and v != [] and v != 0 and v is not None:
        print(f"  {k}: {v}")

assert merged['npc_stats']['button_mother']['trust'] == -2, "npc_stats 병합 실패"
assert merged['npc_status_changes'] == {"button_mother": "sleeping", "button_father": "sleeping"}, "npc_status_changes 병합 실패"
assert "dog_treat" in merged['inventory_add'], "inventory_add 병합 실패"
assert "industrial_sedative" in merged['inventory_remove'], "inventory_remove 병합 실패"
assert merged['vars']['total_suspicion'] == 5, f"vars 누적 실패: {merged['vars']['total_suspicion']}"
print()
print("[PASS] npc_stats 병합 (덮어쓰기가 아닌 누적)")
print("[PASS] npc_status_changes 병합")
print("[PASS] inventory_add/remove 병합")
print("[PASS] vars 누적 병합 (3 + 2 = 5)")

---
## 검증 5: `ToolResult` 스키마 — 반환값 필드 확인

In [ ]:
from app.schemas.tool import ToolResult

print("=== ToolResult 스키마 필드 목록 ===")
for field_name, field_info in ToolResult.model_fields.items():
    print(f"  {field_name}: {field_info.annotation}")

print()
print("=== Tool별 반환값 매핑 ===")
field_usage = [
    ("state_delta",    "모든 tool",  "NPC 스탯, 인벤토리, 상태 변화 등"),
    ("event_description", "모든 tool", "발생한 사건 묘사 리스트"),
    ("intent",         "모든 tool",  "DayController가 채움 (investigate/obey/rebel 등)"),
    ("npc_response",   "interact()만", "NPC가 실제로 한 대사"),
    ("npc_id",         "interact()만", "대화 상대 NPC ID"),
    ("item_id",        "use()만",   "사용한 아이템 ID"),
]

for field, who_uses, description in field_usage:
    exists = field in ToolResult.model_fields
    status = "[PASS]" if exists else "[FAIL]"
    print(f"  {status} {field:20s} ({who_uses:12s}) ← {description}")

In [ ]:
# use() 결과를 ToolResult로 만드는 과정 시뮬레이션 (DayController 내부 로직)

# 위에서 만든 sedative_result를 ToolResult로 변환
from app.schemas.tool import ToolResult

tool_result = ToolResult(
    event_description=sedative_result.get("event_description", []),
    state_delta=sedative_result.get("state_delta", {}),
    intent="rebel",  # DayController가 채워줌
    item_id=sedative_result.get("item_id"),  # use()에서만 존재
)

print("=== ToolResult 생성 검증 (use() 결과) ===")
print(f"event_description: {tool_result.event_description}")
print(f"intent: {tool_result.intent}")
print(f"item_id: {tool_result.item_id}")  # None이면 use()에 item_id가 없는 것
print(f"npc_response: {tool_result.npc_response}  ← use()는 NPC 대사 없음")
print(f"npc_id: {tool_result.npc_id}  ← use()는 NPC ID 없음")
print()
print("state_delta 내 핵심 변화:")
print(f"  npc_status_changes: {tool_result.state_delta.get('npc_status_changes', {})}")
print(f"  inventory_remove: {tool_result.state_delta.get('inventory_remove', [])}")

assert tool_result.item_id == "industrial_sedative", "[FAIL] ToolResult에 item_id가 없음"
assert tool_result.npc_response is None, "[FAIL] use()는 npc_response가 없어야 함"
print()
print("[PASS] ToolResult에 item_id 필드 정상 동작")
print("[PASS] use()는 npc_response 없음 (interact()만 가짐)")

---
## 검증 6: 실패 케이스 검증

아이템 사용이 실패해야 할 경우에 올바르게 실패하는지 확인합니다.

In [ ]:
# 실패 케이스 1: 인벤토리에 없는 아이템 사용
world_empty = copy.deepcopy(world)
world_empty.inventory = []  # 인벤토리 비어있음
set_tool_context(world_state=world_empty, assets=assets, llm_engine=llm_engine)

fail_result = use(item="industrial_sedative", action="수면제를 탄다")

print("=== 실패 케이스 1: 인벤토리에 없는 아이템 ===")
print(f"event_description: {fail_result['event_description']}")
print(f"state_delta: {fail_result['state_delta']}")
iur = fail_result['item_use_result']
print(f"success: {iur['success']}")
print(f"failure_reason: {iur['failure_reason']}")
assert iur['success'] == False
print("[PASS] 인벤토리에 없으면 실패 처리됨")

print()

# 실패 케이스 2: 조건 미충족 (수면제: current_phase가 evening_prep이 아님)
world_wrong_phase = copy.deepcopy(world)
world_wrong_phase.inventory = ["industrial_sedative"]
world_wrong_phase.vars["current_phase"] = "day"  # 낮에는 사용 불가
set_tool_context(world_state=world_wrong_phase, assets=assets, llm_engine=llm_engine)

fail_result2 = use(item="industrial_sedative", action="수면제를 탄다")

print("=== 실패 케이스 2: 조건 미충족 (낮에 수면제 사용) ===")
print(f"event_description: {fail_result2['event_description']}")
iur2 = fail_result2['item_use_result']
print(f"success: {iur2['success']}")
print(f"failure_reason: {iur2['failure_reason']}")
assert iur2['success'] == False
print("[PASS] current_phase != 'evening_prep'이면 수면제 사용 실패")

---
## 최종 요약

검증 결과를 표로 정리합니다.

In [ ]:
print("=" * 70)
print("  Tool 수정 사항 검증 최종 결과")
print("=" * 70)
print()

results = [
    ("action() world_snapshot 구조",
     "전체 world_state 대비 핵심 정보만 추출",
     "_build_world_snapshot() 확인"),
    ("action() 프롬프트 분리",
     "build_action_prompt()가 (system, user) tuple 반환",
     "Tuple[str, str] 반환 타입 확인"),
    ("use() 룰 엔진 기반",
     "LLM 없이 ItemUseResolver로만 동작",
     "ItemUseResolver.resolve() 직접 호출"),
    ("use() inventory_remove",
     "consumable/tool 아이템 소비 시 state_delta에 반영",
     "warm_black_tea 테스트"),
    ("use() npc_status_changes",
     "수면제로 모든 NPC sleeping 상태 변경",
     "industrial_sedative 테스트"),
    ("StatusEffect 생성",
     "duration 있는 상태 효과가 item_use_result.status_effects에 생성",
     "StatusEffect 만료 턴 계산 확인"),
    ("ToolResult.item_id",
     "use() 결과에 item_id 필드 존재",
     "ToolResult 스키마 확인"),
    ("StateDelta 스키마",
     "inventory_add/remove, npc_status_changes 필드 존재",
     "StateDelta.model_fields 확인"),
    ("실패 케이스 처리",
     "인벤토리 미보유 / 조건 미충족 시 올바른 실패 반환",
     "ItemUseResult.success=False 확인"),
]

for i, (title, description, how) in enumerate(results, 1):
    print(f"  {i:2d}. {title}")
    print(f"       → {description}")
    print(f"       검증 방법: {how}")
    print()

print("=" * 70)
print("위 셀들이 모두 [PASS]로 실행됐다면 수정 사항이 올바르게 적용된 것입니다.")
print("=" * 70)